# SVD for Audio Compression

In [1]:
#@title import dependecies
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm

## Representing Audio as Images (*Spectrograms*)

In [2]:
def load_audio(file_path):
    # Carica l'audio
    audio, sr = librosa.load(file_path, sr=None)
    # Riproduci l'audio
    return ipd.Audio(data=audio, rate=sr)

### **Short-Time Fourier Transform (STFT)**

The Short-Time Fourier Transform (STFT) is a fundamental technique in signal processing for analyzing the frequency content of a signal over time. It provides a time-frequency representation of the signal, which is particularly useful for non-stationary signals whose frequency content changes over time.

---

**How STFT Works**

1. **Windowing**:
   - The audio signal is divided into overlapping segments or frames using a window function (e.g., Hamming or Hann window).
   - Each segment is multiplied by the window function to minimize edge effects and to isolate a small portion of the signal.

2. **Fourier Transform**:
   - The Fourier Transform is applied to each windowed segment to convert it from the time domain to the frequency domain.
   - This process results in a complex spectrum for each segment, capturing both amplitude (magnitude) and phase information.

3. **Combining Spectra**:
   - The spectra from all segments are combined to form a 2D representation: the STFT matrix.
   - Each column of the STFT matrix corresponds to the spectrum of a single segment, and each row represents a frequency component over time.

---

**STFT Representation**

- **Magnitude**: The magnitude of the STFT represents the amplitude of different frequency components in each time segment. It is typically visualized as a spectrogram, where the color intensity indicates the amplitude.
- **Phase**: The phase of the STFT captures the phase angle of the frequency components, which is essential for reconstructing the original signal.

---

**References**

- [librosa documentation on stft](https://librosa.org/doc/main/generated/librosa.stft.html)
- Allen, J.B., & Rabiner, L.R. (1977). A unified approach to short-time Fourier analysis and synthesis. *Proceedings of the IEEE*, 65(11), 1558-1564.

In [3]:
def create_spectrogram(file_path):
    # Carica l'audio
    audio, sr = librosa.load(file_path, sr=None)
    # Calcola la STFT (Spettrogramma)
    stft = librosa.stft(audio)
    magnitude = np.abs(stft)
    phase = np.angle(stft)
    return magnitude, phase, sr

# Funzione per visualizzare lo spettrogramma
def plot_spectrogram(magnitude, sr):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.amplitude_to_db(magnitude, ref=np.max), sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spettrogramma')
    plt.tight_layout()
    plt.show()

### **Griffin-Lim Algorithm**

The Griffin-Lim algorithm is a widely used iterative method for phase reconstruction in audio processing. It converts a magnitude spectrogram (i.e., the magnitude of the Short-Time Fourier Transform or STFT) back into an audio waveform. Since the STFT of an audio signal is complex-valued, it includes both magnitude and phase information. However, if only the magnitude is available, the phase needs to be estimated to reconstruct the audio signal. Griffin-Lim iteratively refines the phase estimates to minimize the error between the target magnitude spectrogram and the spectrogram of the reconstructed signal.

---

**How Griffin-Lim Works**

1. **Initialization**:
   - Start with an initial guess of the phase, which is typically random or zeros.
   - Combine the initial phase with the given magnitude to form the initial complex spectrogram.

2. **Iterative Phase Refinement**:
   - In each iteration:
     1. **Inverse STFT**: Perform an inverse Short-Time Fourier Transform (iSTFT) on the current complex spectrogram to get a time-domain signal.
     2. **STFT**: Compute the STFT of the resulting time-domain signal.
     3. **Update Phase**: Replace the magnitude of the STFT with the original magnitude while keeping the new phase.
   - Repeat this process for a specified number of iterations to progressively refine the phase.

3. **Final Audio Reconstruction**:
   - After the iterations, perform a final iSTFT to obtain the reconstructed audio signal.

---

**References**

- Griffin, D., & Lim, J. (1984). Signal estimation from modified short-time Fourier transform. *IEEE Transactions on Acoustics, Speech, and Signal Processing*, 32(2), 236-243.
- [librosa documentation on griffinlim](https://librosa.org/doc/main/generated/librosa.griffinlim.html)

In [11]:
def spectrogram2audio(magnitude, sr, n_iter=32):
    # Ricostruisci l'audio utilizzando l'algoritmo Griffin-Lim
    audio = librosa.griffinlim(magnitude, n_iter=n_iter)
    return audio

def play_from_spectrogram(magnitude, sr, n_iter=32):
    audio = spectrogram2audio(magnitude, sr, n_iter)
    return ipd.Audio(data=audio, rate=sr)

###Example of conversion

In [ ]:
# audio example from the github repo
!wget -O audio.wav 'https://raw.githubusercontent.com/mich1803/SVD-Audio-Compression/main/example%20songs%20to%20convert/Tory%20Lanez%20-%20Lavender%20Sunflower.wav'
filepath = 'audio.wav' #change this if you want to change song

In [ ]:
load_audio(filepath)

In [ ]:
spec, _, sr = create_spectrogram(filepath)
plot_spectrogram(spec, sr)

In [ ]:
play_from_spectrogram(spec, sr)

##SVD Compression

We can compress an image with SVD by decomposing it into three matrices:  $U$ ,  $\Sigma$ , and  $V^T$ . The matrix  $\Sigma$  contains singular values that represent the importance of each corresponding component in  $U$  and  $V^T$ . By keeping only the largest singular values and truncating the smaller ones, we can reconstruct a lower-rank approximation of the original image. This reduces the data size while retaining most of the image’s visual information.

We will apply this technique to spectrograms, which are essentially images representing the frequency spectrum of signals over time.

In [17]:
def svd_compression(spettro, comp):
    """
    Compress a spectrogram using SVD.

    Args:
        spectrogram (numpy.ndarray): The spectrogram to compress.
        num_components (int): Number of singular values to retain.

    Returns:
        u, s, v (numpy.ndarray): Reduced SVD decomposition matrices.
    """
    U, S, Vt = np.linalg.svd(spettro, full_matrices=False)
    U = U[:, :comp]
    S = np.diag(S[:comp])
    Vt = Vt[:comp, :]
    return U, S, Vt

In [18]:
def svd2spectrogram(U, S, Vt):
    """
    Recreate a compressed spectrogram from SVD components.

    Args:
        U (numpy.ndarray): Reduced U matrix.
        S (numpy.ndarray): Reduced diagonal S matrix.
        Vt (numpy.ndarray): Reduced transposed V matrix.

    Returns:
        numpy.ndarray: Compressed spectrogram.
    """
    return np.dot(U, np.dot(S, Vt))

### Testing with different components

In [19]:
comps = [5, 50, 100, 250, 500]
USV = dict([(c, '') for c in comps])
c_spec = dict([(c, '') for c in comps])
for c in tqdm(comps):
  U, S, Vt = svd_compression(spec, c)
  usv = (U, S, Vt)
  USV[c] = usv
  c_spec[c] = svd2spectrogram(U, S, Vt)

100%|██████████| 5/5 [01:08<00:00, 13.70s/it]


In [ ]:
c = 5
print(f"singular values: {c}")
plot_spectrogram(c_spec[c], sr)
play_from_spectrogram(c_spec[c], sr)

In [ ]:
c = 50
print(f"singular values: {c}")
plot_spectrogram(c_spec[c], sr)
play_from_spectrogram(c_spec[c], sr)

In [ ]:
c = 100
print(f"singular values: {c}")
plot_spectrogram(c_spec[c], sr)
play_from_spectrogram(c_spec[c], sr)

In [ ]:
c = 250
print(f"singular values: {c}")
plot_spectrogram(c_spec[c], sr)
play_from_spectrogram(c_spec[c], sr)

In [ ]:
c = 500
print(f"singular values: {c}")
plot_spectrogram(c_spec[c], sr)
play_from_spectrogram(c_spec[c], sr)

### Occupied memory

In [22]:
from os import path
print(f"Size of original audio file: {(path.getsize(filepath)/1024):.2f} KB")
print(f"Size of original spectrogram: {(spec.nbytes/1024):.2f} KB")
for c in USV:
  size = sum([USV[c][i].nbytes for i in range(len(USV[c]))])
  print(f"Size of {c} singular values: {(size/1024):.2f} KB")

Size of original audio file: 28476.22 KB
Size of original spectrogram: 57011.62 KB
Size of 5 singular values: 298.22 KB
Size of 50 singular values: 2991.02 KB
Size of 100 singular values: 6001.56 KB
Size of 250 singular values: 15150.39 KB
Size of 500 singular values: 30789.06 KB


The compression of the audio signal using Singular Value Decomposition (SVD) applied to the spectrogram has demonstrated significant efficiency in reducing data size while retaining audio quality. Specifically, retaining 250 singular values offers a substantial reduction in file size—approximately 26.6% of the original spectrogram size—while achieving near-perfect audio reconstruction. This suggests that SVD is highly effective for compressing audio data with minimal loss of quality, striking a favorable balance between compression and fidelity. As the number of singular values increases, the quality of the reconstructed audio improves, approaching that of the original, but with diminishing returns in compression efficiency beyond 250 values.

### Runtime